<a href="https://colab.research.google.com/github/Eman-Adly/Eman/blob/main/graduation_project_f2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [17]:
file_path = "/content/merged_library_data.csv"
merged_df = pd.read_csv(file_path)

In [18]:
if "User_ID" not in merged_df.columns or "Book_ID" not in merged_df.columns or "Rating" not in merged_df.columns:
    raise ValueError("الملف لا يحتوي على الأعمدة المطلوبة: User_ID, Book_ID, Rating")

In [19]:
merged_df.dropna(subset=["User_ID", "Book_ID", "Rating"], inplace=True)

In [3]:
# تحويل معرفات المستخدمين والكتب إلى أرقام تسلسلية
user_mapping = {user: idx for idx, user in enumerate(merged_df["User_ID"].unique())}
book_mapping = {book: idx for idx, book in enumerate(merged_df["Book_ID"].unique())}
merged_df["User_ID"] = merged_df["User_ID"].map(user_mapping)
merged_df["Book_ID"] = merged_df["Book_ID"].map(book_mapping)

In [4]:
# تعريف Dataset مخصص
class BookDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df["User_ID"].values, dtype=torch.long)
        self.books = torch.tensor(df["Book_ID"].values, dtype=torch.long)
        self.ratings = torch.tensor(df["Rating"].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.books[idx], self.ratings[idx]

In [5]:
# إنشاء DataLoader
dataset = BookDataset(merged_df)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [6]:
# تعريف نموذج NCF
class NCF(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim=8):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.book_embedding = nn.Embedding(num_books, embedding_dim)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, user, book):
        user_embedded = self.user_embedding(user)
        book_embedded = self.book_embedding(book)
        concat = torch.cat([user_embedded, book_embedded], dim=-1)
        rating_pred = self.fc_layers(concat)
        return rating_pred.squeeze()

In [7]:
# تهيئة النموذج
num_users = len(user_mapping)
num_books = len(book_mapping)
model = NCF(num_users, num_books)

In [8]:
# تعريف دالة الفقد والمُحسّن
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [20]:
# تدريب النموذج
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for users, books, ratings in dataloader:
        optimizer.zero_grad()
        predictions = model(users, books)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/10, Loss: 1.6156
Epoch 2/10, Loss: 1.5732
Epoch 3/10, Loss: 1.4854
Epoch 4/10, Loss: 1.4654
Epoch 5/10, Loss: 1.4019
Epoch 6/10, Loss: 1.3181
Epoch 7/10, Loss: 1.2710
Epoch 8/10, Loss: 1.1889
Epoch 9/10, Loss: 1.1389
Epoch 10/10, Loss: 1.0991


In [21]:
# دالة توصية الكتب لمستخدم معين
def recommend_books(user_id, num_recommendations=5):
    if user_id not in user_mapping:
        return "User not found"

    user_idx = torch.tensor(user_mapping[user_id], dtype=torch.long)
    book_indices = torch.tensor(list(book_mapping.values()), dtype=torch.long)
    predictions = model(user_idx.expand(len(book_indices)), book_indices).detach().numpy()

    book_scores = {book: score for book, score in zip(book_mapping.keys(), predictions)}
    recommended_books = sorted(book_scores, key=book_scores.get, reverse=True)[:num_recommendations]

    return merged_df[merged_df['Book_ID'].isin(recommended_books)][["Book_ID", "Title", "Author"]].drop_duplicates()

In [22]:
print(recommend_books(user_id=20))


    Book_ID                   Title        Author
36     B041      The Quest of Quest  Casey Taylor
99     B103  The Shadows of Mystery     Alex Hall
307    B080     The Future of Dream     Quinn Lee
607    B036  The Shadows of Shadows     Riley Lee
623    B181    The Shadows of Dream  Morgan Smith


In [23]:
print(recommend_books(user_id=5))


    Book_ID                   Title          Author
148    B064  The Mystery of Mystery  Skyler Johnson
162    B053     The Future of Dream     Quinn Smith
183    B069   The Journey of Legacy  Quinn Anderson
427    B157    The Dream of Journey    Alex Johnson
607    B036  The Shadows of Shadows       Riley Lee


In [24]:
print(recommend_books(user_id=40))


    Book_ID                     Title         Author
16     B005  The Future of Adventures     Skyler Lee
99     B103    The Shadows of Mystery      Alex Hall
160    B173    The Shadows of Shadows   Skyler Clark
236    B166       The Quest of Future  Taylor Walker
607    B036    The Shadows of Shadows      Riley Lee


In [25]:
print(recommend_books(user_id=30))


    Book_ID                         Title        Author
99     B103        The Shadows of Mystery     Alex Hall
107    B146  The Adventures of Adventures  Taylor Brown
200    B031      The Adventures of Legacy  Morgan Brown
209    B109      The Chronicles of Future   Casey Clark
629    B006  The Chronicles of Chronicles  Skyler Smith
